In [1]:
import os
import subprocess
import numpy as np
import pandas as pd
from astropy.io import fits

In [2]:
# Define the file path (assuming it's in the same directory)
file_name = "final_all_hxmt_lc_periods_chisq_LE.txt"

# Read the file, skipping comment lines (starting with ##)
df = pd.read_csv(file_name, sep='\s+', comment="#")

# Display the first few rows to check
print(df.head())

# Convert obsID column to string (in case it's read as an integer)
df["LC_File_Path"] = df["LC_File_Path"].astype(str)

                                        LC_File_Path   Period  Chi-square  \
0  /home/iist/aniket_data/hxmt_raw_data/decompres...  9.79482     10210.0   
1  /home/iist/aniket_data/hxmt_raw_data/decompres...  9.79473      4372.0   
2  /home/iist/aniket_data/hxmt_raw_data/decompres...  9.79434      4448.0   
3  /home/iist/aniket_data/hxmt_raw_data/decompres...  9.79438      5297.0   
4  /home/iist/aniket_data/hxmt_raw_data/decompres...  9.79446     15670.0   

    Date        freq      PErr  
0  20154  102.092905  0.000022  
1  20154  102.092905  0.000072  
2  20154  102.103329  0.000318  
3  20155  102.103329  0.000070  
4  20155  102.092905  0.000025  


In [ ]:
# Find all obsID folders in "reduced_output"
obsid_folders = [f.path for f in os.scandir("./hxmt_me_l3_products_pipeline/") if f.is_dir() and f.name.startswith("6050")]

# Define nphase values (modify if needed)
nphase = 32  # Or adjust based on requirements
epoch='INDEF'

# Loop through each obsID folder
for obsid_folder in obsid_folders:
    obsid = os.path.basename(obsid_folder)
    print(f"Processing obsID: {obsid}")

    # Find the period for this obsID from the DataFrame
    period_row = df[df["LC_File_Path"] == obsid]
    
    if period_row.empty:
        print(f"No period found for obsID: {obsid}, skipping.")
        continue

    period = period_row["Period"].values[0]

    # Find the correct light curve files
    lc_files = []
    for root, dirs, files in os.walk(obsid_folder):
        for file in files:
            if file.endswith("cl_night_barycorr_orbit_piexpiex.evt"):
                lc_files.append(os.path.join(root, file))

    if not lc_files:
        print(f"No light curve files found for obsID: {obsid}")
        continue

    print(f"Found {len(lc_files)} light curve files for obsID {obsid} with period {period}.")

    # Define output base directory
    output_base = "./folded_curves"
    os.makedirs(output_base, exist_ok=True)  # Create main output folder if not exists
    # Create obsID-specific output folder
    obsid_output_dir = os.path.join(output_base, obsid)
    os.makedirs(obsid_output_dir, exist_ok=True)

    # Run efold for each LC file
    for lc_file in lc_files:
        light_curve_file = lc_file
        output_profile = os.path.join(obsid_output_dir, f"{obsid}_pulse_profile_efold_different_epoch.fits")

        print("runs")

        # Constructing the efold command with all required flags
        cmd = (
            f"echo '-' | efold 1 {lc_file} "
            f"outfile={output_profile} "
            f"dper={period} "
            f"nphase={nphase} "
            f"epochfo=1 "
            f"sepoch={epoch} "
            f"nbint=INDEF "
            f"nintfm=INDEF "
            f"plot=no "
            f"norm=0"
        )

        print(f"Running: {cmd}")
        subprocess.run(cmd, shell=True, check=True)
        break

print("All processing completed!")

Processing obsID: 6050390261
Found 1 light curve files for obsID 6050390261 with period 9.7925.
runs
Running: echo '-' | efold 1 ./reduced_output/6050390261/ni6050390261_0mpu7_cl_night_barycorr_orbit_piexpiex.evt outfile=./folded_curves/6050390261/6050390261_pulse_profile_efold_different_epoch.fits dper=9.7925 nphase=32 epochfo=1 sepoch=INDEF nbint=INDEF nintfm=INDEF plot=no norm=0
 
efold 1.2 (xronos6.0)
 
 Series 1 file    1:./reduced_output/6050390261/ni6050390261_0mpu7_cl_night_bary
 
 Selected FITS extensions: 1 - RATE TABLE; 3 - GTIs
 
 Source ............ Swift_J0243.6+61    Start Time (d) .... 20174 01:46:47.184
 FITS Extension ....  1 - `EVENTS    `   Stop Time (d) ..... 20174 20:33:45.184
 No. of Rows .......      1013196        Bin Time (s) ......   0.4000E-07
 Right Ascension ... 40.9186             Internal time sys.. Converted to TJD
 Declination ....... 61.4323             Experiment ........ NICER    XTI
 
 Corrections applied: Vignetting - No ; Deadtime - No ; Bkgd - N